In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('../../MNIST_data',one_hot=True)

Extracting ../../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
from tensorflow.contrib import rnn

In [3]:
timesteps = 28
n_input = 28
n_class = 10
n_hidden = 128
batch_size = 64

X = tf.placeholder(tf.float32, shape=[None, timesteps, n_input])
Y = tf.placeholder(tf.float32, shape=[None, n_class])

weights = {
    'W': tf.Variable(tf.random_normal([n_hidden, n_class]))
}
biases ={
    'b': tf.Variable(tf.random_normal([n_class]))
}

In [4]:
def RNN(x, weights, biases):
    x = tf.unstack(x, timesteps, 1)
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    outputs,states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    return tf.matmul(outputs[-1], weights['W'])+biases['b']

In [5]:
logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
train = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)

correct_pred = tf.equal(tf.argmax(prediction,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [6]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(1, 10001):
    batch_X, batch_Y = mnist.train.next_batch(batch_size)
    #print(batch_Y.shape)
    batch_X = batch_X.reshape((batch_size, timesteps, n_input))
    sess.run(train, feed_dict={X: batch_X, Y: batch_Y})
    if step % 200 == 0 or step == 1:
        loss, acc = sess.run([loss, accuracy], feed_dict={X: batch_X, Y: batch_Y})
        print("Step" + str(step) + ", Minibatch loss: " + ":.4f".format(loss) + \
             ", Training accuracy: "+ "{:.3f}".format(acc))

Step1, Minibatch loss: :.4f, Training accuracy: 0.078


TypeError: Fetch argument 2.8116684 has invalid type <type 'numpy.float32'>, must be a string or Tensor. (Can not convert a float32 into a Tensor or Operation.)